In [2]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
print('min_payload:%s max_payload: %s' % (min_payload, max_payload))

app = dash.Dash(__name__)

ddc_options = [{'label': 'All Sites', 'value': 'ALL'}]
for site in spacex_df['Launch Site'].unique().tolist():
    ddc_options.append({
        'label': site, 'value': site
    })


# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                               
                                dcc.Dropdown(
                                     id='site-dropdown',
                                     options=ddc_options,
                                     value='ALL',
                                     placeholder='Select a Launch Site here',
                                     searchable=True
                                ),
                                
                                html.Br(),

                               
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
             
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    value=[int(min_payload),int(max_payload)],
                                    marks={
                                        0: '0',
                                        2500: '2500',
                                        5000: '5000',
                                        7500: '7500',
                                        10000: '10000'
                                    },
                                ),

                               
                                html.Div(
                                    dcc.Graph(id='success-payload-scatter-chart')),
                                ])


@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Aggregate the data for all launch sites
        success_df = spacex_df.groupby('Launch Site')['class'].value_counts().unstack().reset_index()
        success_df.fillna(0, inplace=True)  
        success_df['Total'] = success_df[1] + success_df[0]  # Calculate total launches
        fig = px.pie(success_df, names='Launch Site', values='Total', title='Total Success Launches by Site')
    else:
        # Filter data for the selected launch site
        site_df = spacex_df[spacex_df['Launch Site'] == selected_site]['class'].value_counts().reset_index()
        site_df.columns = ['class', 'Count']
        fig = px.pie(site_df, names='class', values='Count', title=f'Success Launches at {selected_site}')
    return fig


@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")]
)
def update_scatter_chart(selected_site, payload_range):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload_range[0],payload_range[1])]

    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]

    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category'
    )

    return fig

if __name__ == '__main__':
    app.run_server(host = "localhost",port = 5000)

min_payload:0.0 max_payload: 9600.0


In [ ]:
pin freeze 